# Abbreviation Disambiguation in Medical Texts - Data Preprocessing

This Notebook is in continuation of the notebook- 'Step 1- Data Wrangling and EDA' and lists down:

1. Data Preprocessing done on the dataset.
2. Exploring the Preprocessed data.

## Step# 1: Load the datasets

In [1]:
# Lets download the spacy library
#!pip install spacy

In [2]:
#Importing the Required Python Packages
import os
import shutil
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_colwidth', -1)

In [3]:
# Lets load the default english model of spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
# Lets load the train dataset.
train = pd.read_csv('Data/train.csv')

In [5]:
#Lets check the dataset
train.head()

,ABSTRACT_ID,TEXT,LOCATION,LABEL
0,14145090,velvet antlers vas are commonly used in traditional chinese medicine and invigorant and contain many PET components for health promotion the velvet antler peptide svap is one of active components in vas based on structural study the svap interacts with tgfÎ² receptors and disrupts the tgfÎ² pathway we hypothesized that svap prevents cardiac fibrosis from pressure overload by blocking tgfÎ² signaling SDRs underwent TAC tac or a sham operation T3 one month rats received either svap mgkgday or vehicle for an additional one month tac surgery induced significant cardiac dysfunction FB activation and fibrosis these effects were improved by treatment with svap in the heart tissue tac remarkably increased the expression of tgfÎ² and connective tissue growth factor ctgf ROS species C2 and the phosphorylation C2 of smad and ERK kinases erk svap inhibited the increases in reactive oxygen species C2 ctgf expression and the phosphorylation of smad and erk but not tgfÎ² expression in cultured cardiac fibroblasts angiotensin ii ang ii had similar effects compared to tac surgery such as increases in Î±smapositive CFs and collagen synthesis svap eliminated these effects by disrupting tgfÎ² IB to its receptors and blocking ang iitgfÎ² downstream signaling these results demonstrated that svap has antifibrotic effects by blocking the tgfÎ² pathway in CFs,63,transverse aortic constriction
1,1900667,the clinical features of our cases demonstrated some of the already known characteristics of the variable spectrum of hiv infection da are the most important risk category in italy of the arc cases evolved into aids during a month followup on average the most frequent oi in our aids cases were pcp c albicans esophagitis and chronic mucocutaneous ulcers an high percentage of neurologic involvement from hiv was observed and HM were encountered in aids ks and undifferentiated b lymphoma as well as in arc HD statistically significant worsening of the immunologic situation is evident as the disease progresses from las to aids G1 b lymphocytes represent most of the cells of the germinal center during the hyperplastic stage of lymphadenopathy reversal of the tt ratio appears early during the initial stage of lymphadenopathy and is due to a decrease of cd and a relative increase of cd also destruction of the follicular dendritic cells is an early feature which becomes more evident as the disease advances and the lymph node evolves toward progressive involution activated blymphocyte augmentation with polyclonal ig secretion appears to be related to TI b stimulation by coinfection such as cmv ebv and hbv the increase of CD8 lymphocytes seems to be partly related to the excessive activation of b lymphocytes and partially directed to the cells INF by hiv or coated with its proteins the destruction of follicular dendritic cells has been interpreted not only as a killer effect of the virus but also as a result of the intervention of ctl sensitized to the cells containing the virus their destruction may contribute to the impaired recognition of soluble antigen which is one of the main features of the immune deficiency of hiv infection,85,hodgkins lymphoma
2,8625554,ceftobiprole bpr is an investigational cephalosporin with activity against staphylococcus aureus including methicillinresistant s aureus mrsa strains the pharmacodynamic pd profile of bpr against s aureus strains with a variety of susceptibility phenotypes in an immunocompromised murine pneumonia model was characterized the bpr mics of the test isolates ranged from to mugml pharmacokinetic pk studies were conducted with infected neutropenic balbc mice and the bpr concentrations were measured in plasma epithelial lining fluid elf and lung tissue pd studies with these mice were undertaken with eight s aureus isolates two MSSA strains three hospitalacquired mrsa strains and three CA mrsa strains subcutaneous bpr doses of to mgkg of body weightday were administered an

## Step# 2a: Create a new feature 'ABV'

In [6]:
# Lets create a function to create a new feature 'ABV' from dataset
def createFeature(df):    
    return [x.split(' ')[y] for x,y in zip(df['TEXT'], df['LOCATION'])]

In [7]:
train['ABV'] = createFeature(train)

### Lets check which Abbreviations occur the most and lets take top 20 such Abbreviations for further processing due to Hardware limitations

In [8]:
grouped = train.groupby(by=['ABV', 'LABEL'], as_index = False, sort = False).count()
grouped = grouped.sort_values(by='TEXT', ascending = False)
grouped

,ABV,LABEL,ABSTRACT_ID,TEXT,LOCATION
12111,FL,fulllength,268,268,268
4972,HA,hepatic artery,268,268,268
4969,CAs,catecholamines,268,268,268
4968,beta,human fibroblast,268,268,268
4967,tau,circadian period,268,268,268
...,...,...,...,...,...
22286,PTB,patellar tendon bearing,6,6,6
12192,ATA,thermolabile antigen,6,6,6
22294,L-NMMA,lnmonomethyl arginine,6,6,6
21742,FPC,fish protein concentrate,6,6,6


In [9]:
#Lets take top 20 Abbreviations for further processing
topAbv = grouped['ABV'][:20]
topAbv

12111    FL  
4972     HA  
4969     CAs 
4968     beta
4967     tau 
4965     AC  
13223    HR  
4963     SNs 
4961     BP  
4959     TD  
9918     V1  
4955     AD  
4952     GH  
4951     MN  
4949     GP  
4948     AE  
4943     IR  
9922     EZ  
15345    WD  
4937     ca  
Name: ABV, dtype: object

### Lets extract only the above 20 Abbreviations from train set for futher processing.

In [10]:
train = train[train['ABV'].isin(topAbv)]
train.shape

(52579, 5)

## Step# 2: Data Preprocessing: Following preprocessing steps will be performed on the Dataset:

    a. Create a new feature 'ABV' for abbreviation directly deriving it from Location and Text columns.-- Already done above
    b. Convert text to lowercase.
    c. Remove Punctuations from the Text Column.
    d. Tokenize the Text column.
    e. Dropping Abstract_id, Location and Text columns.
    f. Remove stop words from the Text Column.

So, lets start with the Data Preprocessing sub-steps.

## Step# 2b: Convert the data to lowercase

In [11]:
#Lets create a function to convert all the text in lowercase
def tolower(df):
    return [t.lower() for t in df['TEXT']]

## Step# 2c: Remove Punctuations

In [12]:
# Lets create a function to remove all the Punctuations from Text
def removePunctuation(df):
    return [t.translate(str.maketrans('','',string.punctuation)) for t in df['TEXT']]

## Step# 2d: Tokenize the text column and save the tokenized data in a new column 'TOKEN'.

In [13]:
# Lets create a function to Tokenize the Text column of dataset
def createTokens(df):
    return df['TEXT'].apply(lambda x: x.split(' '))

## Step# 2e: Dropping the columns not needed

In [15]:
#Lets create a function to drop "Abstract_id", "Location" and "TEXT" columns from dataset
def dropCols(df):
    return df.drop(columns=['ABSTRACT_ID', 'LOCATION', 'TEXT'])

## Step 2f: Remove Stop words

In [16]:
# Lets create a function to remove stop words from the Text column
def removeStop(df):
    stopWords = spacy.lang.en.stop_words.STOP_WORDS
    # Remove any stopwords which appear to be an Abbreviation
    [stopWords.remove(t) for t in df['ABV'].str.lower() if t in stopWords]
    return df['TOKEN'].apply(lambda x: [item for item in x if not item in stopWords])

## Lets Create a function to apply all the above preprocessing steps to the dataset

In [18]:
def preProcessData(df):   
    df['TEXT'] = tolower(df)
    df['TEXT'] = removePunctuation(df)
    df['TOKEN'] = createTokens(df)
    df = dropCols(df)
    df['TOKEN'] = removeStop(df)
    return df

### Apply the above preProcessData function to train set

In [19]:
preProcessData(train).to_csv('Train/train_final.csv', index = False)

### Lets load the Validation and Test set for Preprocessing.

In [20]:
# Lets load the Valid and test datasets as well.
valid = pd.read_csv('Data/valid.csv')
test = pd.read_csv('Data/test.csv')

In [21]:
valid.head(3)

,ABSTRACT_ID,TEXT,LOCATION,LABEL
0,5017844,different electrocardiographic changes have been described during thrombolytic therapy for AIM to indicate successful reperfusion the occluded coronary artery also can be reopened by percutaneous TCA ptca this study was performed to compare electrocardiographic changes during primary or rescue ptca and thrombolytic therapy the electrocardiographic changes were studied directly at the moment of reperfusion during ptca,25,transluminal coronary angioplasty
1,8923525,there are limited data regarding cdx expression in rectal carcinoma the ckck immunoprofile of CRC has been described in studies which have mostly lumped Tc and rectal PT together in this study we investigated the diagnostic utility of immunohistochemical stains for ck ck and cdx in a series of rectal adenocarcinoma fiftyfive specimens of rectal adenocarcinomas were retrieved and immunostained for ck dakom ck novocastra ncllck and cdx novocastra nclcdx thirty cases of pancreatic adenocarcinoma and CC were also studied as a comparison group ck was expressed in and ck in cases of rectal adenocarcinoma the ckck immunophenotype was identified in ckck in and ckck in rectal adenocarcinoma cdx showed moderatestrong positivity in all cases and was not related to RT differentiation benign rectal mucosa was available in cases and showed the following results ckck in ckck in and ckck in cases in pancreatic adenocarcinomas and cholangiocarcinomas were ckck and were ckck cdx was positive in only of these cases all were pancreatic adenocarcinomas in conclusion ck can be expressed in rectal adenocarcinoma and should not be used as the sole basis for excluding a rectal primary cdx is a CS marker for rectal origin of adenocarcinoma it can be helpful in cases with metastatic rectal carcinoma especially those with ckck or ckck immunophenotype in this study cdx expression was not influenced by the grade differentiation of rectal adenocarcinoma,14,colorectal adenocarcinoma
2,4675370,many healthcare facilities and other notforprofit organizations employ enough people to be covered by the new worker adjustment and retraining notification act which requires businesses to give days notice of a shutdown or mass layoffs leaders of these organizations should be aware of the laws terms the law requires that any business enterprise that employs fulltime workers give days notice of a closing or mass layoffs it also covers businesses that employ or more workers both full time and part time who work at least a total of hours per week it defines employer as any business enterprise that employs the required number of persons interim interpretive rules include NFP organizations in this definition and the final rules are not likely to change this the organization must give notice in writing to each representative of the affected employees or to each employee in addition it must notify in writing the state dislocated worker unit and the local government to which the business paid the highest taxes in the previous year notice is not required if it would prevent the organization from obtaining capital or business necessary to continue operations if the shutdown is due to NK disaster or if the shutdown was unforeseeable the law also allows exceptions for strikes lockouts and temporary jobs,109,notforprofit


In [22]:
test.head(3)

,ABSTRACT_ID,TEXT,LOCATION,LABEL
0,2069316,we developed an animal model of chronic allergic airway disease by repeatedly exposing nine sheep to tracheal instillation of ascaris antigen until stable increase in RL at three times control in six reactive sheep group c was obtained they were then compared to the three nonreactive sheep group b and a control group of eight sheep exposed to saline only group a in terms of pulmonary CF tests and bronchoalveolar lavage bal analyses RL was cm holsec in group a in group b and in group c trapping volume FRC by plethysmography and by helium rebreathing technique was l in group a in group b and in group c UP resistance at PF did not differ between any two CG but UP resistance near residual volume was cm holsec in a in b and in c in bal total cells were x ml in a in b and in c macrophages in bal were in a in b and in c neutrophils were in a in b in c eosinophils were in a in b and in c p less than group c versus group a total proteins albumin ALP phosphatase and fibronectin did not differ between groupsabstract truncated at words,89,functional residual capacity
1,6967959,pyogenic granulomas represent the aquisition of vasodilative granulation tissue in the skin or mucosa they are extremely rare in the alimentary tract other than in the oral cavity here we report a case of PG arising from the GM an yearold man was admitted to our hospital because of melena of more than months duration esophagogastroduodenoscopy egd revealed a mmdiameter semipedunculated lesion with an irregular surface in the fundus of the stomach during hospitalization the patients anemia worsened due to loss of blood from the lesion with the level of hemoglobin declining to gdl and a blood transfusion was required because radiological and endoscopic findings indicated the lesion was hypervascular transarterial embolization of the nutritional i.a. of the lesion was performed before ER of the lesion one week T3 the embolotherapy endoscopic mucosal resection was performed without any complications such as massive one histological studies of the resected specimen revealed many capillaries of various sizes lined with plump EC cells and accompanied by acute and chronic inflammatory infiltrates on the basis of these observations the lesion was diagnosed as a PG one year later the patient was asymptomatic and there was no evidence of RT recurrence on followup egd,34,pyogenic granuloma
2,3492721,the l immunotype lipopolysaccharide lps of neisseria meningitidis was subjected to Kd procedures which produced a number of different oligosaccharide fragments the high resolution h and c nmr spectroscopic analyses of these OS yielded structural information on a number of different regions of the lps for example from one oligosaccharide it was found that the endogenous sialylation of the meningococcal lps occurs at o of the terminal betadgalactopyranosyl residue of its lactonneotetraose antenna in the alphadconfiguration from another it was also established that the dominant structural feature responsible for l epitope specificity is the presence of a PE substituent at o of the penultimate heptopyranosyl residue of its other antenna in addition from information obtained with another oligosaccharide the structure of the deoxydmannooctulosonic acid disaccharide region of the l lps was also elucidated from all the above cumulative data plus some published data it was then possible to reconstruct the CR structure of the entire native l lps,97,phosphorylethanolamine


### Let's only use the Abbreviations for which we are training the model

In [23]:
# Create ABV feature for valid and test sets.
valid['ABV'] = createFeature(valid)
test['ABV'] = createFeature(test)

In [24]:
# Filter the valid and test datasets based on the topAbv list and check their shapes
valid = valid[valid['ABV'].isin(topAbv)]
test = test[test['ABV'].isin(topAbv)]
print('Valid:', valid.shape)
print('Test:', test.shape)

Valid: (17526, 5)
Test: (17515, 5)


### Again, due to Hardware limitations, lets use 10K rows for validation and test sets.

In [25]:
valid = valid[:10000]
test = test[:10000]

### Lets apply the preprocessing steps to Valid and Test datasets

In [26]:
valid = preProcessData(valid)

In [27]:
test = preProcessData(test)

In [28]:
valid.to_csv('Validation/valid_final.csv', index = False)

In [29]:
test.to_csv('Test/test_final.csv', index = False)